# Import

In [1]:
import os
import re
import gc
import sys
from loguru import logger

import matplotlib.pyplot as plt 

from datetime import datetime

import numpy as np
import torch


sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from HETSFileHelper import gatherCSV, readChannel, EIS_recal_ver02
from Outlier import OutlierDetection
from EISGPR import Interpolation


# %matplotlib qt

# Filesys

In [2]:
def SearchELE(rootPath, ele_pattern = re.compile(r"(.+?)_归档")):
    '''==================================================
        Search all electrode directories in the rootPath
        Parameter: 
            rootPath: current search path
            ele_pattern: electrode dir name patten
        Returen:
            ele_list: list of electrode directories
        ==================================================
    '''
    ele_list = []
    for i in os.listdir(rootPath):
        match_ele = ele_pattern.match(i)
        if match_ele:
            ele_list.append([os.path.join(rootPath, i),match_ele.group(1)])
    return ele_list

In [3]:
# rootPath = "D:/Baihm/EISNN/Archive/"
# ele_list = SearchELE(rootPath)

rootPath = "D:/Baihm/EISNN/Invivo/"
ele_list = SearchELE(rootPath, re.compile(r"(.+?)_Ver01"))

n_ele = len(ele_list)
logger.info(f"Search in {rootPath} and find {n_ele:03d} electrodes")

2025-04-11 11:34:15.980 | INFO     | __main__:<module>:8 - Search in D:/Baihm/EISNN/Invivo/ and find 005 electrodes


# Error Processed Statistic

In [ ]:
# 我们观察到，由于我们在最后聚类的时候使用了AP + silhouette_score
# 而silhouette_score 对最低样本数有要求
# 这使得我们会遇到大量报错，之前用try exception跳过了，但是这个可能会导致我们把正常电极误判
# 这里我们打印每个pt文件中，有效电极数和追踪天数
# 如果有效电极数 < 128 - 10 且追踪天数比较多，就认为有问题

MODEL_SUFFIX = "Matern12_Ver01"

n_miss = 0
n_perfect = 0
n_good = 0
n_bad = 0
n_terrible = 0
n_error = 0

n_perfect_ch  = 0
n_good_ch = 0
n_bad_ch = 0
n_terrible_ch = 0
n_error_ch = 0

n_eval_day_sum = 0
n_eval_times_sum = 0

for i in range(n_ele):
# for i in range(3):
    # logger.info(f"ELE Begin: {ele_list[i][0]}")
    fd_pt = os.path.join(ele_list[i][0], MODEL_SUFFIX, f"{ele_list[i][1]}_{MODEL_SUFFIX}.pt")
    if not os.path.exists(fd_pt):
        n_miss = n_miss + 1
        logger.warning(f"{fd_pt} does not exist")
        continue
    data_pt = torch.load(fd_pt)
    _meta_group = data_pt["meta_group"]
    _data_group = data_pt["data_group"]

    n_day       = _meta_group["n_day"]
    n_ch        = _meta_group["n_ch"]
    n_valid_ch  = len(_data_group["Channels"])

    # n_eval_day = _data_group['ch_000']['x_eval'].max()
    # print(n_eval_day)

    if n_ch != 128:
        n_error = n_error + 1
        logger.critical(f"{ele_list[i][1]}[{i:03d}]: {n_valid_ch}/{n_ch} [{n_day}]")
    elif n_valid_ch != n_ch:
        if n_day > 4 and n_valid_ch>100:
            n_good = n_good + 1
            n_good_ch = n_good_ch + n_valid_ch
            n_eval_day_sum = n_eval_day_sum + _data_group[_data_group["Channels"][0]]['x_eval'].max() * n_valid_ch
            n_eval_times_sum = n_eval_times_sum + n_day * n_valid_ch


            logger.debug(f"{ele_list[i][1]}[{i:03d}]: {n_valid_ch}/{n_ch} [{n_day}]")
        else:
            if n_day > 4:
                n_bad = n_bad + 1
                n_bad_ch = n_bad_ch + n_valid_ch
                logger.warning(f"{ele_list[i][1]}[{i:03d}]: {n_valid_ch}/{n_ch} [{n_day}]")
            else:
                n_terrible = n_terrible + 1
                n_terrible_ch = n_terrible_ch + n_valid_ch
                logger.error(f"{ele_list[i][1]}[{i:03d}]: {n_valid_ch}/{n_ch} [{n_day}]")
    else:
        n_perfect = n_perfect + 1
        n_perfect_ch = n_perfect_ch + 128
        n_eval_day_sum = n_eval_day_sum + _data_group[_data_group["Channels"][0]]['x_eval'].max() * n_valid_ch
        n_eval_times_sum = n_eval_times_sum + n_day * n_valid_ch

        logger.info(f"{ele_list[i][1]}[{i:03d}]: {n_valid_ch}/{n_ch} [{n_day}]")


C:\Users\Baihm\AppData\Local\Temp\1\ipykernel_2212\2298384240.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_pt = torch.load(fd_pt)
2025-04-11 11:34:16.070 | WARN

In [5]:

logger.info(f"\nn_miss:{n_miss}\nn_perfect:{n_perfect}\nn_good:{n_good}\nn_bad:{n_bad}\nn_error:{n_error}\nn_terrible:{n_terrible}\n{n_miss+n_perfect+n_good+n_bad+n_terrible+n_error}")
logger.info(f"\nn_perfect_ch:{n_perfect_ch}\nn_good_ch:{n_good_ch}\
            \nn_bad_ch:{n_bad_ch}\nn_terrible_ch:{n_terrible_ch}\
            \nn_eval_day_sum:{n_eval_day_sum}\nn_eval_day_avg:{n_eval_day_sum/(n_perfect_ch+n_good_ch)}\
            \nn_eval_times_sum:{n_eval_times_sum}\nn_eval_times_avg:{n_eval_times_sum/(n_perfect_ch+n_good_ch)}\
            ")


2025-04-11 11:34:16.776 | INFO     | __main__:<module>:1 - 
n_miss:0
n_perfect:0
n_good:3
n_bad:2
n_error:0
n_terrible:0
5
2025-04-11 11:34:16.777 | INFO     | __main__:<module>:2 - 
n_perfect_ch:0
n_good_ch:379            
n_bad_ch:79
n_terrible_ch:0            
n_eval_day_sum:27077.0
n_eval_day_avg:71.44327176781003            
n_eval_times_sum:3665
n_eval_times_avg:9.67018469656992            
